# Hamblin semantics introductory fragment
## Author: Kyle Rawlins
### Updated: 7/11/16

This is an implementation of Hamblin semantics.  It is currently quite incomplete, but walks through the basic principles.  It also may be useful for describing some lexicon manipulation techniques in the lambda notebook (see also the appendix).

In [ ]:
reload_lamb()
%lambctl reset

In [ ]:
type_e = types.type_e
type_t = types.type_t
type_n = types.type_n
type_s = types.BasicType("s")
ts = meta.get_type_system()
ts.add_atomic(type_s)
ts

The first thing to do is to 'hamblinize' the lexicon.  This amounts to converting ordinary meanings to singleton sets with their contents.

There are a number of options for how to implement this in a typed lambda calculus.  Here, we will exploit the fact that the Lambda Notebook has a type for sets.  If $\alpha$ is a type, then $\{\alpha\}$ is the type of sets containing elements of type $\alpha$.

The next bit of code defines a special operation `hamblinize` to simplify the process a bit; it can be safely ignored.

In [ ]:
def hamblinize_te(te):
    """Hamblinize a single lexical item.  Do so by building a singleton set out of it."""
    if meta.get_type_system().unify(te.type, meta.tp("{X}")): 
        # assume this item is already hamblinized
        return te
    elif meta.get_type_system().unify(te.type, meta.tp("<{X},Y>")): 
        # heuristic: if function whose domain is a set of some sort, assume that this is a Hamblin operator.
        # may miss cases.  Better to just run this on content items...
        return te
    # wrap the content of the lexical item as a singleton set.
    return meta.ListedSet([te])    

#continuize_lex(cat1.content)
lamb.parsing.eq_transforms["hamblin"] = hamblinize_te

In [ ]:
hamblinize_te(te("L x: Cat(x)"))

Let's see this in operation:

In [ ]:
%%lamb
||cat|| =<hamblin> L x_e : L w_s : Cat(w,x)
||gray|| =<hamblin> L x_e : L w_s : Gray(w,x)
||john|| =<hamblin> John_e
x =<hamblin> L y_e : y
||test|| = L x_e : Test(x) # don't hamblinize this

Next, we need to add a way of composing entries *pointwise*.  The following code implements the most general case of Pointwise FA.

In the long run, one would want to automatically simplify these expressions; this is easy to do by hand, but it is a hard problem in the general case, so things are left mostly unsimplified.

In [ ]:
pfa_combinator = %te L f_{<X,Y>} : L a_{X} : Set x_Y : Exists f1_<X,Y> : (Exists a1_X : (f1 << f & a1 << a) & x <=> f1(a1))
pfa_combinator

In [ ]:
system = lang.td_system.copy()
system.add_binary_rule(pfa_combinator, "PFA")
lang.set_system(system)
system

In [ ]:
john * cat

Instead of automatically simplifying things, I'll exploit the fact that sets are represented differently (as a list) if they are finite, so we can special-case PFA for ListedSets, and fall back on the combinator for the general case.

Note that some limited reduction does occur with the $\in$ operator; $x \in \{y \:|\: \phi\}$ gets converted to $(\lambda y : \phi)(x)$ and reduced, illustrated in the next cell.

In [ ]:
r = te("x_e << (Set y_e : Test_<e,t>(y))")
r.reduce_all().derivation

In [ ]:
def pfa_listed(fun, arg):
    result = list()
    for felem in fun.args:
        for aelem in arg.args:
            result.append(felem(aelem))
    return meta.ListedSet(result)

def pfa_general(fun, arg):
    ts = meta.get_type_system()
    general = pfa_combinator(fun)(arg) # do this for type checking
    if isinstance(fun, meta.ListedSet) and isinstance(arg, meta.ListedSet):
        return pfa_listed(fun, arg)
    else:
        return general.reduce_all()
    
system = lang.td_system.copy()
system.add_binary_rule_uncurried(pfa_general, "PFA")
lang.set_system(system)
system

In [ ]:
john * cat

In [ ]:
%%lamb
## To simplify, let's take there to only be three human-like entities in the universe.
||who|| = {John_e, Mary_e, Sue_e}
||saw|| =<hamblin> L x_e : L y_e : L w_s : Saw(w,y,x)

In [ ]:
(cat * who).tree()

Wh-items compose in situ.  Intuitively, this leads to sets that percolate up the tree.

In [ ]:
john * (saw * who)

In [ ]:
(john * (saw * who)).tree()

In [ ]:
who * (saw * john)

The analysis of existentials in a compositional Hamblin semantics (e.g. for Japanese) can be illustrated in this simple example by adding Hamblin 1-place quantificational operators that compose via ordinary FA.

In [ ]:
%%lamb
||HExists|| = L p_{<s,t>} : {(Lambda w_s  : Exists q_<s,t> : q(w) & (q << p))}
||HForall|| = L p_{<s,t>} : {(Lambda w_s  : Forall q_<s,t> : q(w) >> (q << p))}

In [ ]:
HExists * (who * (saw * john))

In [ ]:
HExists * (john * (saw * who))

In [ ]:
(HExists * (john * (saw * who))).tree()

More generally, the listed versions of this won't do, because we don't want to assume that the relevant sets are finite.  Or in other words, we don't want to define 'who' extensionally:

In [ ]:
%%lamb
||who|| = Set x_e : Human(x)

At the moment the lambda notebook doesn't do much simplification of these expressions, so they will be a bit unwieldy.

In [ ]:
(cat * who).tree()

In [ ]:
john * (saw * who)

In [ ]:
who * (saw * john)

In [ ]:
HExists * (who * (saw * john))

The above formulas are somewhat involved, so it is slightly more convenient to simply define a toy example with a finite set of individuals:

### Appendix: another technique for Hamblinization

Another way of Hamblinizing a lexicon would be to write extra magics for converting whole lexicons at once.  Here's a sketch of how to do this.

In [ ]:
def hamblinize_item(item):
    """Hamblinize a single lexical item.  Do so by building a singleton set out of it."""
    if meta.ts_compatible(item.type, meta.tp("{?}")): #isinstance(item.type, types.SetType):
        # assume this item is already hamblinized
        return item
    elif meta.ts_compatible(item.type, meta.tp("<{?},?>")): #item.type.functional() and isinstance(item.type.left, types.SetType):
        # heuristic: if function whose domain is a set of some sort, assume that this is a Hamblin operator.
        # may miss cases.  Better to just run this on content items...
        return item
    new_i = item.copy()
    # wrap the content of the lexical item as a singleton set.
    new_i.content = meta.ListedSet([item.content])
    return new_i

# in the following two magics, variables that are not lexical items are ignored.  To change this, modify the else case above.
def h_magic(self, accum):
    """Hamblinize the accumulated definitions from a single cell, as a post-processing step"""
    new_accum = lamb.magics.process_items(hamblinize_item, accum)[0]
    for k in new_accum.keys():
        self.env[k] = new_accum[k]
    return new_accum

def h_magic_env(self):
    """Hamblinize the entire env"""
    self.env = lamb.magics.process_items(hamblinize_item, self.env)[0] # hamblinize every variable
    self.shell.push(self.env) # export the new variables to the interactive shell
    return lamb.parsing.latex_output(self.env, self.env)

lamb.magics.LambMagics.specials_post["hamblinize"] = h_magic
lamb.magics.LambMagics.specials["hamblinize_all"] = h_magic_env

In [ ]:
%%lamb reset,hamblinize
||cat|| = L x_e : L w_s : Cat(w,x)
||gray|| = L x_e : L w_s : Gray(w,x)
||john|| = J_e
x = L y_e : y

In [ ]:
%%lamb
||test|| = L x_e : Test(x)

In [ ]:
%lambctl hamblinize_all